블록 4개, ABD, ABC를 블록과 중첩시켜라.

실험 결과:

|처리조합|I|처리조합|I|
|---|---|---|---|
|$(1)$|$90$|$d$|$98$|
|$a$|$74$|$ad$|$72$|
|$b$|$81$|$bd$|$87$|
|$ab$|$83$|$abd$|$85$|
|$c$|$77$|$cd$|$99$|
|$ac$|$81$|$acd$|$79$|
|$bc$|$88$|$bcd$|$87$|
|$abc$|$73$|$abcd$|$80$|

# 효과 추정

In [10]:
import pandas as pd
import statsmodels.api as sm

df = pd.DataFrame({
    'Treament': ['(1)', 'a','b','ab','c','ac','bc','abc','d','ad','bd','abd','cd','acd','bcd','abcd'],
})

df['a'] = df['Treament'].map(lambda x: 1 if 'a' in x else -1)
df['b'] = df['Treament'].map(lambda x: 1 if 'b' in x else -1)
df['c'] = df['Treament'].map(lambda x: 1 if 'c' in x else -1)
df['d'] = df['Treament'].map(lambda x: 1 if 'd' in x else -1)
df['ABC_Block'] = df[['a','b','c']].prod(1)
df['ABD_Block'] = df[['a','b','d']].prod(1)
df['Block'] = [1,2,2,1,3,4,4,3,4,3,3,4,2,1,1,2]
df['Y'] = [90,74,81,83,77,81,88,73,98,72,87,85,99,79,87,80]
df

,Treament,a,b,c,d,ABC_Block,ABD_Block,Block,Y
0,(1),-1,-1,-1,-1,-1,-1,1,90
1,a,1,-1,-1,-1,1,1,2,74
2,b,-1,1,-1,-1,1,1,2,81
3,ab,1,1,-1,-1,-1,-1,1,83
4,c,-1,-1,1,-1,1,-1,3,77
5,ac,1,-1,1,-1,-1,1,4,81
6,bc,-1,1,1,-1,-1,1,4,88
7,abc,1,1,1,-1,1,-1,3,73
8,d,-1,-1,-1,1,-1,1,4,98
9,ad,1,-1,-1,1,1,-1,3,72


In [19]:
from DOE import effect_est

for fac in df['Treament']:
    if fac == '(1)': continue
    if fac in ['abc', 'abd']: 
        print(f'Effect of {fac.upper()}Block:',effect_est(data=df, sign_col=[f for f in fac], value_cols=['Y']))
    else:
        print(f'Effect of {fac.upper()}:',effect_est(data=df, sign_col=[f for f in fac], value_cols=['Y']))

Effect of A: -10.0
Effect of B: -0.75
Effect of AB: 4.5
Effect of C: -0.75
Effect of AC: 0.5
Effect of BC: -1.25
Effect of ABCBlock: -6.0
Effect of D: 5.0
Effect of AD: -3.75
Effect of BD: -1.5
Effect of ABDBlock: 4.75
Effect of CD: 1.5
Effect of ACD: -0.25
Effect of BCD: -2.0
Effect of ABCD: 3.25


# 분산분석

In [23]:
model = sm.OLS.from_formula(formula='Y ~ a + b + c + d + C(Block)', data=df).fit()
model.summary()

c:\Users\USER\miniconda3\envs\doe_env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     4.031
Date:                Wed, 04 Dec 2024   Prob (F-statistic):             0.0345
Time:                        16:19:53   Log-Likelihood:                -43.375
No. Observations:                  16   AIC:                             102.7
Df Residuals:                       8   BIC:                             108.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        84.7500      2.574     32.927      0.000      78.815      90.685
C(Block)[T.2]    -1.2500      3.640     -0.343      0.740      -9.644       7.144
C(Block)[T.3]    -7.5000      3.640     -2.060      0.073     -15.894       0.894
C(Block)[T.4]     3.2500      3.640      0.893      0.398      -5.144      11.644
a                -5.0000      1.287     -3.885      0.005      -7.968      -2.032
b                -0.3750      1.287     -0.291      0.778      -3.343       2.593
c                -0.3750      1.287     -0.291      0.778      -3.343       2.593
d                 2.5000      1.287      1.943      0.088      -0.468       5.468
==============================================================================
Omnibus:                        1.398   Durbin-Watson:                   2.622
Prob(Omnibus):                  0.497   Jarque-Bera (JB):                1.102
Skew:                           0.584   Prob(JB):                        0.576
Kurtosis:                       2.462   Cond. No.                         4.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
sm.stats.anova_lm(model, typ=3).round(4)

,sum_sq,df,F,PR(>F)
Intercept,28730.25,1.0,1084.1604,0.0000
C(Block),243.25,3.0,3.0597,0.0915
a,400.00,1.0,15.0943,0.0046
b,2.25,1.0,0.0849,0.7782
c,2.25,1.0,0.0849,0.7782
d,100.00,1.0,3.7736,0.0880
Residual,212.00,8.0,NaN,NaN
